In [4]:
import pandas as pd
import torch
from transformers import pipeline

In [2]:
df_train = pd.read_excel(r'C:\Users\John\Desktop\Patent Datasets\final_data\data_train_final.xlsx')

# Checking the dataset

In [4]:
column_to_check = 'claim_x'

In [5]:
df_train['WordCount'] = df_train[column_to_check].apply(lambda x: len(str(x).split()))


In [6]:
max_word_count_row = df_train.loc[df_train['WordCount'].idxmax()]

print(f"The row with the highest number of words is:\n{max_word_count_row}")

The row with the highest number of words is:
title_x              AUTONOMOUS SUSTAINABLE WIND UNIT, MULTI-BLADE ...
description_x        A.—INTRODUCTION This patent application has fo...
citation_category                                                    A
title_y              Wind-powered energy production and storing system
claim_y              1. A wind-powered energy production and storin...
claim_x              1 . Autonomous sustainable wind unit, reticula...
description_y        The invention relates to a wind-powered energy...
WordCount                                                         3856
Name: 19475, dtype: object


In [17]:
# Specify the column and row index
column_name = 'description_y'
row_index = 1

# Check the length of the content in the specified cell
cell_length = len(df_train.at[row_index, column_name])
print(f"The length of the content in cell ({row_index}, '{column_name}') is: {cell_length}")

The length of the content in cell (1, 'description_y') is: 32767


In [7]:
specific_column = 'claim_x'
row_index = 19475

# Fetch the specific cell value
try:
    value = df_train.loc[row_index, specific_column]  # Use loc for label-based indexing
    print(f"Value at row {row_index} of column '{specific_column}': {value}")
except KeyError:
    print(f"Row {row_index} not found in the DataFrame.")

Value at row 19475 of column 'claim_x': 1 . Autonomous sustainable wind unit, reticular multi-blade rotor, accumulator and energy converter and applications which consists of memory, claim and figures and is claimed as “a solidarity set” of an “autonomous energy unit”, operating as a generating source, accumulator, transformer and accumulator and distributor of thermal and dynamic energy, continuously, composed, in the wind field by a wind turbine, with a horizontal axis rotor, consisting of radial reticular trusses, square metal hollow welded and braced bars that support in their extreme sections both self-regulating, simple, double, aerodynamic blades, etc., articulated with a certain eccentricity so that the blades tend to rotate an angle (a), depending on the wind speed, in the direction of the side with the largest surface, at the same time that the deformation reaction of a spring, arranged in each blade, counteracts it until reaching a of equilibrium before the action of the nom

In [8]:
average = df_train['WordCount'].mean()

print(f"The average of column is: {average}")

The average of column is: 123.78461423723758


In [10]:
# Count how many values are above 300
count_above_300 = (df_train['WordCount'] > 400).sum()

print(f"The number of values in column above 300 is: {count_above_300}")

The number of values in column above 300 is: 379


# LLAMA IMPLEMENTATION

In [3]:
def create_sliding_windows(text, window_size, step_size):
    """
    Creates sliding windows from a given text.

    Args:
        text (str): The input text to divide into sliding windows.
        window_size (int): The number of words in each window.
        step_size (int): The number of words to move the window by.

    Returns:
        list: A list of sliding windows (subtexts).
    """
    words = text.split()  # Split the text into words
    total_words = len(words)
    windows = []

    for i in range(0, total_words - window_size + 1, step_size):
        window = " ".join(words[i:i + window_size])
        windows.append(window)

    return windows

In [8]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

In [9]:
# Initialize the text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,  # If you have GPU support
    device_map="auto",  # Automatically uses available GPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
target_desc = df_train['description_y'][0]
source_claim = df_train['claim_x'][0]

In [7]:
window_size = 500  # Number of words in each window
step_size = 50    # Number of words to move the window by

sliding_windows = create_sliding_windows(target_desc, window_size, step_size)

# Print sliding windows
for i, window in enumerate(sliding_windows, 1):
    print(f"Window {i}: {window}")

Window 1: CROSS-REFERENCE TO A RELATED APPLICATION The invention described and claimed hereinbelow is also described in German Patent Application DE 10 2006 012 008.6 filed on Mar. 14, 2006. This German Patent Application, whose subject matter is incorporated here by reference, provides the basis for a claim of priority of invention under 35 U.S.C. 119(a)-(d). BACKGROUND OF THE INVENTION The present invention relates to an electrohydraulic control unit for rotor blade adjustment of a wind farm. One such electrohydraulic control unit is known from German Utility Model DE 203 17 749. In wind farms, rotor blades are adjustable about a rotor blade axis within a predetermined angle range, so that the rotor blade can be positioned into the wind (known as “pitching out”) or positioned relative to the wind direction (known as “pitching in”). Since rotor blades are very expensive and vulnerable, and because of the strong forces that prevail especially in a strong wind, a rotor blade adjustment 

In [20]:
# Define a function to split the long description into manageable chunks
def split_text(text, max_tokens):
    """Split the input text into chunks with a maximum token length."""
    words = text.split()
    chunks = []
    current_chunk = []

    # Accumulate words until we reach the token limit
    for word in words:
        current_chunk.append(word)
        if len(current_chunk) > max_tokens:
            chunks.append(" ".join(current_chunk[:-1]))  # Add current chunk (excluding last word)
            current_chunk = [current_chunk[-1]]  # Start a new chunk with the last word
    chunks.append(" ".join(current_chunk))  # Add the last chunk

    return chunks

In [ ]:
# Split the long patent description into smaller chunks
description_chunks = split_text(large_text, max_tokens=512)

In [10]:
# Create the instruction template
instruction_template = """
Based on the following claim from Patent 1, generate a new patent claim that incorporates the relevant features mentioned in the description of Patent 2.

Claim from Patent 1: {claim}

Description from Patent 2: {chunk}

Print only the new claim with no notes or comments, just a raw claim with 450 words max. If you don't find any relevant features ignore the current chunk. 
"""


In [16]:
# Split the long patent description into smaller chunks
window_size = 500  # Number of words in each window
step_size = 50    # Number of words to move the window by

sliding_windows = create_sliding_windows(target_desc, window_size, step_size)

# Process each chunk and generate new claims
generated_claims = []
for chunk in sliding_windows:
    instruction = instruction_template.format(claim=source_claim, chunk=chunk)
    outputs = pipe(instruction, max_new_tokens=500)  # Limit output length to prevent excessive text
    generated_text = outputs[0]["generated_text"]

    # Post-process the output to remove the instruction part
    new_claim = generated_text.replace(instruction, "").strip()

    # Append only the new claim to the list
    generated_claims.append(new_claim)

# Combine the generated claims from each chunk
final_claim = "\n".join(generated_claims)
print(final_claim)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id

A wind turbine (1) having a rotor (4) with at least two blades (5) and a blade pitch system (6) for controlling the pitch angle of said blades (5), 
the blade pitch system (6) comprising for each of said blades a hydraulic blade pitch drive (8) and at least two valves (22a, 22b, 22c) mutually connected in parallel for controlling a flow of liquid to the hydraulic blade pitch drive (8) for that blade (5), wherein said valves (22a, 22b, 22c) each comprises an arrangement for providing a variable flow of liquid and wherein said at least two valves are proportional hydraulic spool valves (22a, 22b, 22c), 
the rotor (4) having a hydraulic cylinder (9) with a piston chamber (10) and a piston rod chamber (11), 
an inflow-side valve assembly (12) for establishing a pressure fluid connection between a pump (13) and the piston chamber (10), 
an outflow-side valve assembly (14) for establishing a pressure fluid connection between the piston rod chamber (11) and a tank (15), 
said inflow valve ass

In [ ]:
# Process each chunk and generate new claims
generated_claims = []
for chunk in description_chunks:
    instruction = instruction_template.format(claim=claim_patent_1, chunk=chunk)
    outputs = pipe(instruction, max_new_tokens=500)  # Limit output length to prevent excessive text
    generated_text = outputs[0]["generated_text"]

    # Try to remove the instruction and description from the generated output
    # Assuming that the output is a continuation of the instruction, we will strip it
    new_claim = generated_text.replace(instruction, "").strip()  # Remove the instruction part

    # Append only the new claim to the list
    generated_claims.append(new_claim)

# Combine the generated claims from each chunk
final_claim = "\n".join(generated_claims)
print(final_claim)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A wind turbine (1) having a rotor (4) with at least two blades (5) and a blade pitch system (6) for controlling the pitch angle of said blades (5), the blade pitch system (6) comprising for each of said blades a hydraulic blade pitch drive (8) and at least two valves (22a, 22b, 22c) mutually connected in parallel for controlling a flow of liquid to the hydraulic blade pitch drive (8) for that blade (5), wherein said valves (22a, 22b, 22c) each comprises an arrangement for providing a variable flow of liquid and wherein said at least two valves are proportional hydraulic spool valves (22a, 22b, 22c) that can be switched in different combinations to control the pitch angle of said blades (5), and wherein the hydraulic blade pitch drive (8) is connected to a pressure line that communicates with the piston chamber of a hydraulic cylinder and with a tank, and wherein the piston chamber of the hydraulic cylinder is connected to a pump and to the outflow-side valve assembly, and wherein the o

In [27]:
df_train['claim_x'][0]

'A wind turbine (1) having a rotor (4) with at least two blades (5) and a blade pitch system (6) for controlling the pitch angle of said blades (5), \nthe blade pitch system (6) comprising for each of said blades a hydraulic blade pitch drive (8) and at least two valves (22a, 22b, 22c) mutually connected in parallel for controlling a flow of liquid to the hydraulic blade pitch drive (8) for that blade (5), wherein said valves (22a, 22b, 22c) each comprises an arrangement for providing a variable flow of liquid and wherein said at least two valves are proportional hydraulic spool valves (22a, 22b, 22c).'